<a href="https://colab.research.google.com/github/niapollard/Benchmarking_QC/blob/main/052024_Al_Varies%20Reps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q jarvis-tools qiskit==0.43.1

In [9]:
!pip install qiskit_nature==0.6.2

In [10]:
!pip install pyscf

In [11]:
!pip install qiskit-aer==0.12.0

In [12]:
import numpy as np

# Importing Qiskit and Jarvis libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers.second_quantization import MethodType
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.transformers.second_quantization.electronic.active_space_transformer import ActiveSpaceTransformer
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
import itertools, functools
from qiskit.opflow import I, X, Y, Z
from jarvis.db.figshare import get_wann_electron, get_wann_phonon, get_hk_tb
from jarvis.core.circuits import QuantumCircuitLibrary
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from pyscf import gto, scf, tools

In [13]:
#Build the molecule
molecule = Molecule(
    geometry=[['Al',[0.00000000, 0.00000000, 0.00000000]]],
    charge=-1,
    multiplicity=1)

In [14]:
def get_qubit_op(
    molecule,
    basis='sto3g',
    functional='lda',
    method= MethodType.RKS,
    driver_type= ElectronicStructureDriverType.PYSCF,
    converter= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]
  mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)

  res1={}
  res1['qubit_op']=qubit_op
  res1['converter']=converter
  res1['problem_reduced']=problem_reduced
  return res1 #qubit_op, converter, problem_reduced

def get_energy(optimizer='',device='',qubit_op='',seed='', reps=1):
  seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=1).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['sto3g']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
#converter=[JordanWignerMapper()]
functionals=['lda']
devices=[Aer.get_backend('statevector_simulator')]
reps_list = [1, 2, 3, 4]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          #for n in converter:
            for o in functionals:
              for p in devices:
                for reps in reps_list:
                  print(i,j,k,m,o,p,reps)

                  res1=get_qubit_op(molecule=molecule,basis=j,functional=o,method=k)
                  res=get_energy(optimizer=m,device=p,qubit_op=res1['qubit_op'],seed=42)
                  print(res['eigenvalue'])

                  #GroundStateEigensolver
                  solver =GroundStateEigensolver(res1['converter'], res['vqe'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  mem[i,j,k,m,o,p,reps]= {
                      'eigenvalue': res['eigenvalue'],
                      'vqe': res['vqe'],
                      'qi': res['qi'],
                      'converter': res1['converter'],
                      'problem_reduced': res1['problem_reduced']
                     }

#print(mem)

Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7fddf2c22b30> lda statevector_simulator 1


<ipython-input-14-84e3d1675a19>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-14-84e3d1675a19>:59: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-14-84e3d1675a19>:62: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -238.842309564845
  - computed part:      -1.677391036755
  - ActiveSpaceTransformer extracted energy part: -237.16491852809
~ Nuclear repulsion energy (Hartree): 0.0
> Total ground state energy (Hartree): -238.842309564845
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.977 S: 0.491 S^2: 0.733 M: 0.488
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-0.000191  0.00049908  -0.00039418]
    - computed part:      [-0.000191  0.00049908  -0.00039418]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.000191  -0.00049908  0.00039418]  Total: 0.00066403
                 (debye): [0.00048547  -0.00126853  0.00100191]  Total: 0.00168781
 
Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7fddf2c22b30> lda statevector_simulator 2


...

(8.5066017987e-08+0j)
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -238.842309565452
  - computed part:      -1.677391037814
  - ActiveSpaceTransformer extracted energy part: -237.164918527638
~ Nuclear repulsion energy (Hartree): 0.0
> Total ground state energy (Hartree): -238.842309565452
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.977 S: 0.491 S^2: 0.733 M: 0.488
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-0.00019103  0.00049915  -0.00039427]
    - computed part:      [-0.00019103  0.00049915  -0.00039427]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.00019103  -0.00049915  0.00039427]  Total: 0.00066415
                 (debye): [0.00048555  -0.00126872  0.00100214]  Total: 0.0016881
 
Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7fddf2c22b30> lda statevector_simulator 4


<ipython-input-14-84e3d1675a19>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-14-84e3d1675a19>:59: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-14-84e3d1675a19>:62: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -238.842309564318
  - computed part:      -1.677391035359
  - ActiveSpaceTransformer extracted energy part: -237.164918528959
~ Nuclear repulsion energy (Hartree): 0.0
> Total ground state energy (Hartree): -238.842309564318
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.977 S: 0.491 S^2: 0.733 M: 0.488
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-0.00043561  -0.00049686  0.00006328]
    - computed part:      [-0.00043561  -0.00049686  0.00006328]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.00043561  0.00049686  -0.00006328]  Total: 0.0006638
                 (debye): [0.0011072  0.00126289  -0.00016083]  Total: 0.0016872
 
